In [1]:
from data_loader import DataLoader, FriendEdgeEncoding

In [2]:
test_data_loader = DataLoader()
test_data_loader.load_random_train_test_network()
test_data_loader.load_stratified_user_degree_train_test_network()
test_data_loader.load_stratified_user_train_test_network()



In [3]:
test_data_loader = DataLoader(friendship_edge_encoding=FriendEdgeEncoding.NONE, user_embeddings = [], game_embeddings = ['name'], user_game_edge_embeddings = ['example_sum_user_id_game_id_playtime_forever'], friend_friend_edge_embeddings = [])
network = test_data_loader.get_full_network()
print('Num edges: ', len(network.edges))
print('Num nodes: ', len(network.nodes))
print(list(network.edges(data=True))[:10])
print(list(network.edges(data=True))[-10:])
print(list(network.nodes(data=True))[:10])
print(list(network.nodes(data=True))[-10:])

Num edges:  39544
Num nodes:  9481
[(76561197963485711, 10, {'example_sum_user_id_game_id_playtime_forever': 76561197963485721}), (76561197963485711, 20, {'example_sum_user_id_game_id_playtime_forever': 76561197963485731}), (76561197963485711, 30, {'example_sum_user_id_game_id_playtime_forever': 76561197963485741}), (76561197963485711, 40, {'example_sum_user_id_game_id_playtime_forever': 76561197963485751}), (76561197963485711, 50, {'example_sum_user_id_game_id_playtime_forever': 76561197963485761}), (76561197963485711, 60, {'example_sum_user_id_game_id_playtime_forever': 76561197963485771}), (76561197963485711, 70, {'example_sum_user_id_game_id_playtime_forever': 76561197963485781}), (76561197963485711, 130, {'example_sum_user_id_game_id_playtime_forever': 76561197963485841}), (76561197963485711, 2990, {'example_sum_user_id_game_id_playtime_forever': 76561197963488701}), (76561197963485711, 550, {'example_sum_user_id_game_id_playtime_forever': 76561197963486261})]
[(76561199605560768,

In [4]:
test_data_loader = DataLoader(friendship_edge_encoding=FriendEdgeEncoding.BETWEEN_USERS, user_embeddings = [], game_embeddings = ['name'], user_game_edge_embeddings = ['example_sum_user_id_game_id_playtime_forever'], friend_friend_edge_embeddings = [])
network = test_data_loader.get_full_network()
print('Num edges: ', len(network.edges))
print('Num nodes: ', len(network.nodes))
print(list(network.edges(data=True))[:10])
print(list(network.edges(data=True))[-10:])
print(list(network.nodes(data=True))[:10])
print(list(network.nodes(data=True))[-10:])

Num edges:  45550
Num nodes:  9481
[(76561197963485711, 10, {'example_sum_user_id_game_id_playtime_forever': 76561197963485721}), (76561197963485711, 20, {'example_sum_user_id_game_id_playtime_forever': 76561197963485731}), (76561197963485711, 30, {'example_sum_user_id_game_id_playtime_forever': 76561197963485741}), (76561197963485711, 40, {'example_sum_user_id_game_id_playtime_forever': 76561197963485751}), (76561197963485711, 50, {'example_sum_user_id_game_id_playtime_forever': 76561197963485761}), (76561197963485711, 60, {'example_sum_user_id_game_id_playtime_forever': 76561197963485771}), (76561197963485711, 70, {'example_sum_user_id_game_id_playtime_forever': 76561197963485781}), (76561197963485711, 130, {'example_sum_user_id_game_id_playtime_forever': 76561197963485841}), (76561197963485711, 2990, {'example_sum_user_id_game_id_playtime_forever': 76561197963488701}), (76561197963485711, 550, {'example_sum_user_id_game_id_playtime_forever': 76561197963486261})]
[(76561199605560768,

In [5]:
test_data_loader = DataLoader(friendship_edge_encoding=FriendEdgeEncoding.ALL_FRIENDSHIPS, user_embeddings = [], game_embeddings = ['name'], user_game_edge_embeddings = ['example_sum_user_id_game_id_playtime_forever'], friend_friend_edge_embeddings = [])
network = test_data_loader.get_full_network()
print('Num edges: ', len(network.edges))
print('Num nodes: ', len(network.nodes))
print(list(network.edges(data=True))[:10])
print(list(network.edges(data=True))[-10:])
print(list(network.nodes(data=True))[:10])
print(list(network.nodes(data=True))[-10:])

Num edges:  60800
Num nodes:  24450
[(76561197963485711, 10, {'example_sum_user_id_game_id_playtime_forever': 76561197963485721}), (76561197963485711, 20, {'example_sum_user_id_game_id_playtime_forever': 76561197963485731}), (76561197963485711, 30, {'example_sum_user_id_game_id_playtime_forever': 76561197963485741}), (76561197963485711, 40, {'example_sum_user_id_game_id_playtime_forever': 76561197963485751}), (76561197963485711, 50, {'example_sum_user_id_game_id_playtime_forever': 76561197963485761}), (76561197963485711, 60, {'example_sum_user_id_game_id_playtime_forever': 76561197963485771}), (76561197963485711, 70, {'example_sum_user_id_game_id_playtime_forever': 76561197963485781}), (76561197963485711, 130, {'example_sum_user_id_game_id_playtime_forever': 76561197963485841}), (76561197963485711, 2990, {'example_sum_user_id_game_id_playtime_forever': 76561197963488701}), (76561197963485711, 550, {'example_sum_user_id_game_id_playtime_forever': 76561197963486261})]
[(76561199605560768